In [5]:
%load_ext autoreload
%autoreload 2

import jax.numpy as jnp
import numpy as np
import optax
import random
import gym
import jax.random as jrandom
from jax import jacobian 
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
env = gym.make('InvertedPendulum-v4')

/Users/damso/Year4/FYP/pipps/venv/lib/python3.7/site-packages/gym/utils/env_checker.py:145: UserWarning: WARN: Agent's minimum observation space value is -infinity. This is probably too low.
  "Agent's minimum observation space value is -infinity. This is probably too low."
/Users/damso/Year4/FYP/pipps/venv/lib/python3.7/site-packages/gym/utils/env_checker.py:149: UserWarning: WARN: Agent's maxmimum observation space value is infinity. This is probably too high
  "Agent's maxmimum observation space value is infinity. This is probably too high"
/Users/damso/Year4/FYP/pipps/venv/lib/python3.7/site-packages/gym/utils/env_checker.py:201: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  "We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) "


In [3]:
def cost_function(state):
    target = jnp.array([0., 0., 1.])
    state_val = jnp.array([state[0], jnp.sin(state[1]), jnp.cos(state[1])])
    diff = state_val - target
    width = 1.
    return 1 - jnp.exp(- ((diff[0] ** 2 + diff[1] ** 2 + diff[2] ** 2) / width) ** 2)

In [7]:
def rollout_episode(theta, env, horizon):
    cur_state = env.reset()
    trajectory = [] 
    for t in range(horizon):
        action = cur_state.T @ theta
        prev_state = cur_state
        cur_state, _, done, info = env.step(np.asarray(action))
        trajectory.append(prev_state)

    loss = jnp.sum([cost_function(state) for state in trajectory])
    return loss

In [8]:
theta = jnp.full((4,), 1.)
params = jnp.array([theta])

optimizer = optax.chain(
    optax.adam(learning_rate=0.005),
    optax.scale(-1.0)
)

opt_state = optimizer.init(params)

Horizon = 20
MAX_TRAJECTORIES = 500
gamma = 0.99
score = [] 

for episode in range(MAX_TRAJECTORIES):

    grads = jacobian(rollout_episode, 0)(params[0], env, Horizon)
    updates, opt_state = optimizer.update(grads, opt_state, params)
    params = optax.apply_updates(params, updates)

    if episode % 10 == 0:
        score = 0
        cur_state = env.reset()
        for t in range(50):
            action = cur_state.T @ params[0]
            prev_state = cur_state
            cur_state, _, done, info = env.step(action)
            if done:
                break
            score += 1
        
        print(f"Step {episode}, score: {score}")
    
theta = params[0]

TracerArrayConversionError: The numpy.ndarray conversion method __array__() was called on the JAX Tracer object Traced<ConcreteArray(0.0025407851208001375, dtype=float32)>with<JVPTrace(level=2/0)> with
  primal = DeviceArray(0.00254079, dtype=float32)
  tangent = Traced<ShapedArray(float32[])>with<JaxprTrace(level=1/0)> with
    pval = (ShapedArray(float32[]), None)
    recipe = JaxprEqnRecipe(eqn_id=<object object at 0x7fa37a5ada90>, in_tracers=(Traced<ConcreteArray([ 0.00726358  0.00082922 -0.00400576 -0.00154626], dtype=float32):JaxprTrace(level=1/0)>, Traced<ShapedArray(float32[4]):JaxprTrace(level=1/0)>), out_tracer_refs=[<weakref at 0x7fa35d9a3ad0; to 'JaxprTracer' at 0x7fa35cda7770>], out_avals=[ShapedArray(float32[])], primitive=xla_call, params={'device': None, 'backend': None, 'name': 'jvp(matmul)', 'donated_invars': (False, False), 'inline': True, 'keep_unused': False, 'call_jaxpr': { lambda ; a:f32[4] b:f32[4]. let
    c:f32[] = dot_general[
      dimension_numbers=(((0,), 